Ok, let's check capabilities of SkLearn befere using NNs

Which method to use? Of course ones that enlisted [here](https://github.com/lemikhovalex/DA_CV) (repo for ML classes at MIPT)
## Preprocess data
But before using data let's create features and labels

The following features are to be created, accoarding to papers about LSTM:
- $t_{w}$ - time passed between Sunday midnight and the event
- $t_{e}$ - time passed between the completion of the given event and the completion of the previous one
- $t_{t}$ - time passed between the start of the trace and the given event
- one-hot-encoded labels
- make smth like n-gramm, but for process and use it as features

### Test train split
Ok it seems to be rational not to take random transitions, but whole traces, cause `n-gramm` preprocessing will be conducted natively, if data is provided as a sequence

In [1]:
import pandas as pd
import datetime
import random

In [2]:
df = pd.read_csv('datasets/bpi_12.csv')
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.datetime.fromisoformat(x))

In [3]:
df

,timestamp,trace_id,activity
0,2011-10-01 00:38:44.546000+02:00,173688,A_SUBMITTED
1,2011-10-01 00:38:44.880000+02:00,173688,A_PARTLYSUBMITTED
2,2011-10-01 00:39:37.906000+02:00,173688,A_PREACCEPTED
3,2011-10-01 11:42:43.308000+02:00,173688,A_ACCEPTED
4,2011-10-01 11:45:09.243000+02:00,173688,O_SELECTED
...,...,...,...
164501,2012-03-10 12:50:54.881000+01:00,214373,W_Nabellen offertes
164502,2012-02-29 23:51:16.799000+01:00,214376,A_SUBMITTED
164503,2012-02-29 23:51:17.423000+01:00,214376,A_PARTLYSUBMITTED
164504,2012-03-01 09:27:37.118000+01:00,214376,A_DECLINED


Also dont want to deal with short traces - composing features as window will fail it(

In [4]:
traces = list(set(df['trace_id']))
print(f'There are {len(traces)} traces')
traces_len = {trace_id: df[df['trace_id'] == trace_id].shape[0] for trace_id in traces}
traces_time = {}
for trace_id in traces:
    _df = df[df['trace_id'] == trace_id]
    if _df.shape[0] > 7:
        traces_time[trace_id] = (_df['timestamp'].max() - _df['timestamp'].min()).total_seconds()
traces_time = pd.DataFrame({'trace_id': list(traces_time.keys()), 'tt': list(traces_time.values())})
traces_time.sort_values(by=['tt'], inplace=True)

There are 13087 traces


In [5]:
traces_time.head()

,trace_id,tt
2941,210755,507.914
3755,180547,708.375
4674,185461,789.907
2455,176392,896.478
6251,196428,953.681


Let's calculate 4 equivalent quantiles 4 trace length distribution and sample 30% from each

In [6]:
n_qs = 4
sh = traces_time.shape[0]
qs = [0.]
qs.extend([traces_time[_i * sh // n_qs: (_i + 1) * sh // n_qs]['tt'].max() for _i in range(n_qs)])

In [7]:
test_ids = []
test_relative = 0.25
for _i in range(n_qs):
    _lb_cond = (traces_time['tt'] > qs[_i])
    _rb_cond = (traces_time['tt'] <= qs[_i+1])
    _cond = _lb_cond & _rb_cond
    _pop = traces_time[_cond]['trace_id'].values.tolist()
    test_ids.extend(random.sample(population=_pop,
                                  k=int(len(_pop) * test_relative)
                                 )
                   )

In [8]:
print(f'There are {len(test_ids)} traces for test')

There are 1568 traces for test


In [9]:
train_ids = traces
for _test_id in test_ids:
    train_ids.remove(_test_id)

In [10]:
print(f'There are {len(train_ids)} traces for train')

There are 11519 traces for train


In [11]:
df.head()

,timestamp,trace_id,activity
0,2011-10-01 00:38:44.546000+02:00,173688,A_SUBMITTED
1,2011-10-01 00:38:44.880000+02:00,173688,A_PARTLYSUBMITTED
2,2011-10-01 00:39:37.906000+02:00,173688,A_PREACCEPTED
3,2011-10-01 11:42:43.308000+02:00,173688,A_ACCEPTED
4,2011-10-01 11:45:09.243000+02:00,173688,O_SELECTED


In [14]:
test_df = df[df['trace_id'].isin(test_ids)]
train_df = df[df['trace_id'].isin(train_ids)]

In [15]:
train_df.to_csv('datasets/train_df.csv')
test_df.to_csv('datasets/test_df.csv')